### Example of how to use the LDSAnalysis class
(tested on Windows 10 with Python v3.10)

In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os

from analysis import LDSAnalysis
from plotting import plot_divergence_curves

load_dotenv('.env')  # Make sure this exists and has the correct path to the data 'PATH_DATA_ROOT'

#### Data must be structured as follows:
PATH_DATA_ROOT
├── P001
│   ├── PRE
│   │   ├── foot_right.csv
│   │   ├── thigh_left.csv
│   │   ├── ...
│   ├── POST
│   │   ├── foot_right.csv
│   │   ├── ...
├── P002
│   ├── ...

#### Files must be structured as follows:
- The data files must be in CSV format
- 3 columns: 'x', 'y', 'z' for the gyroscope data
- The first row must be the header (column names)
- The next 10,000 rows are the data points
- The data points are 100 consecutive strides, resampled to 10,000 data points

In [ ]:
PATH_DATA_ROOT = os.environ.get('PATH_DATA_ROOT')
if PATH_DATA_ROOT is None:
    raise ValueError("The PATH_DATA_ROOT environment variable must be set.")
path_data_in = Path(PATH_DATA_ROOT).joinpath('processed')
path_data_out = Path(PATH_DATA_ROOT).joinpath('analysis')

In [ ]:
sensor_location = 'foot_right'

In [ ]:
analysis = LDSAnalysis(path_data_in=path_data_in,
                       path_data_out=path_data_out,
                       sensor_location=sensor_location)

#### Print a summary of the analysis object

In [ ]:
analysis.summary()

Store the signal data in a pandas DataFrame (pickle file) for time efficiency.
This is automatically done when running the successive steps of the analysis.

In [ ]:
analysis.make_signals()

Compute the time delay for the signal data based on the minimum of the average mutual information. The results for all trials are stored in the `time_delays` (pandas.DataFrame) property of the analysis object.

In [ ]:
analysis.compute_time_delays()  # trigger the computation of the time delay
analysis.time_delay_summary()  # print a summary for reporting and debugging
tau = analysis.time_delay  # gets the average time delay over all participants, conditions, and axes
print(f'{tau=}')

Compute the embedding dimension for the signal data with the given time delay. If the call to `compute_time_delays` wasn't made, it's automatically performed by this method. The results for all trials are stored in the `embedding_dimensions` (pandas.DataFrame) property of the analysis object.

In [ ]:
analysis.compute_embedding_dimensions()  # trigger the computation of the embedding dimension
analysis.embedding_dimension_summary()  # print a summary for reporting and debugging
dE = analysis.embedding_dimension  # gets the maximum embedding dimension over all participants and conditions
print(f'{dE=}')

Reconstruct the state spaces of the signals given the computed time delay and embedding dimension. If the calls to `compute_time_delays` and `compute_embedding_dimensions` weren't made, they're automatically performed by this method. The results for all trials are stored in the `state_spaces` (pandas.DataFrame) property of the analysis object.

In [ ]:
analysis.compute_state_spaces()  # trigger the computation of the state spaces
state_spaces = analysis.state_spaces  # gets the state spaces for all participants and conditions

Compute the divergence curves for the state spaces. Automatically performs the previous steps if they weren't already done. The results for all trials are stored in the `divergence_curves` (pandas.DataFrame) property of the analysis object.

In [ ]:
analysis.compute_divergence_curves()  # trigger the computation of the divergence curves
divergence_curves = analysis.divergence_curves  # gets the divergence curves for all participants and conditions

Use the divergence curves object and pass it to the  for the state spaces. The plot is saved in the `path_data_out` directory.

In [ ]:
plot_divergence_curves(divergence_curves=analysis.divergence_curves,
                       path_data_out=analysis.path_data_out,
                       location_string=analysis.sensor_location)

Detect the region of rapid divergence and determine the number of sample to fit the line through for the LDS. Then set it via the `set_fit_interval()` method. Now, the maximum divergence exponent can be computed. The results are automatically saved in an excel file.

In [ ]:
analysis.set_fit_interval(end=30)  # start=0 per default
analysis.compute_divergence_exponents()
print(analysis.divergence_exponents.head())